# Load Autoencoder and configs

In [1]:
import os
#import time
#import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import yaml
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model


from Autoencoder_Architecture import autoencoder_512x512
from Autoencoder_Architecture import upsample    
from Autoencoder_Architecture import downsample
from Autoencoder_Architecture import Autoencoder
from Autoencoder_Architecture import evaluate
from Autoencoder_Architecture import generate_images
from Autoencoder_Architecture import folder_to_dataloader

def load_hyperparameters(filepath):
    with open(filepath, 'r') as file:
        hyperparameters = yaml.safe_load(file)
    return hyperparameters

filepath = 'config.yaml'
hyperparams = load_hyperparameters(filepath)

# needed config parameters
BEST_CONFIGURATION_ONLY = hyperparams['BEST_CONFIGURATION_ONLY']
MODUS= hyperparams['MODUS']
BEST_CONFIGURATION = hyperparams['BEST_CONFIGURATION']
N_EPOCHS = hyperparams['N_EPOCHS']  
MODEL_NUMBER = hyperparams['MODEL_NUMBER']

In [2]:
configurations = []

for batch_size in [1]:
    for learning_rate, beta_1 in [(2e-4, 0.5),(1e-4, 0.5),(2e-4, 0.9)]: # [(2e-4, 0.5), (1e-5, 0.5),(2e-4, 0.9),(1e-5, 0.9)]:
        for use_bias_term in [True]:
            for use_l2_loss in [True]: # only MSE before [False,True]
                for DROPOUT_VALUE in [0.2,0.5,0.6]: # before only 0.5
                        for KERNEL_SIZE in[3,4]:
                            for WEIGHT_DECAY in [0, 0.004]: # default tensorflow adamW
                                for use_skip_connections in [True]:
                                    batches_to_plot = round(128 / batch_size)
                                    configurations.append({
                                        'BATCH_SIZE': batch_size,
                                        'USE_L2_LOSS': use_l2_loss,
                                        'N_EPOCHS': N_EPOCHS,
                                        'BATCHES_TO_PLOT': batches_to_plot,
                                        'USE_BIAS_TERM': use_bias_term,
                                        'LEARNING_RATE': learning_rate,
                                        'BETA_1': beta_1,
                                        'USE_SKIP_CONNECTIONS': use_skip_connections,
                                        'DROPOUT_VALUE': DROPOUT_VALUE,
                                        'KERNEL_SIZE':KERNEL_SIZE,
                                        'WEIGHT_DECAY': WEIGHT_DECAY,
                                    })

if BEST_CONFIGURATION_ONLY == True:
    configurations= [configurations[BEST_CONFIGURATION]]
else:  
    print(f"Use all Configurations")


In [3]:
for config_idx, configuration in enumerate(configurations):
    print(configuration)
    model = Autoencoder(MODEL_NUMBER=MODEL_NUMBER, configuration=configuration)

{'BATCH_SIZE': 1, 'USE_L2_LOSS': True, 'N_EPOCHS': 10, 'BATCHES_TO_PLOT': 128, 'USE_BIAS_TERM': True, 'LEARNING_RATE': 0.0002, 'BETA_1': 0.5, 'USE_SKIP_CONNECTIONS': True, 'DROPOUT_VALUE': 0.5, 'KERNEL_SIZE': 4, 'WEIGHT_DECAY': 0}


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 2  0           []                               
                                )]                                                                
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 256, 256, 2)  0           ['input_1[0][0]']                
                                                                                                  
 sequential (Sequential)        (None, 128, 128, 64  2112        ['tf.math.truediv[0][0]']        
                                )                                                                 
                                                                                              

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model)


## Load the model weights: only use the filename until epochXX nothing more.

Example: #checkpoint_path = r'C:\Users\burckhardsv\Lokale_Dateien\results\ditzingen_gaussian_20000img_10epochs_13_05_2024\checkpoints\autoencoder\epoch_10'

the it loads th right one data and index.

In [ ]:
checkpoint_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Results\Final_Ditzingen\Real_Vortex_Abb08_best_config_Max_Epochs\Dataset_Vortex_cleaned_shuffeled_best_config_abb08_ditzingen\2024-07-03_20-53-21-303516\checkpoints\autoencoder\epoch_47'
#print(checkpoint_path)
model.load_weights(checkpoint_path)

In [ ]:
#model.get_weights()

# One Image with generation of the images

What we need is a folder with the same structure as the training structure:

- beam_ff: defocus image
- beam_nf: focus image 
- phasemask some random phasemask

In [ ]:
def predict(predict_data, autoencoder,configuration):
    """Predict the Phasemask and give out all metrics and the overview image"""
    final_test_performance = {}
    for subset in ['predict_512']:
        autoencoder_loss, l1_loss, l2_loss, ssim_score = evaluate(autoencoder, predict_data[subset],configuration)
        for metric in ['autoencoder_loss','l1_loss', 'l2_loss', 'ssim_score']:
            final_test_performance[metric] = eval(metric)
        generate_images('final-performance', autoencoder, predict_data[subset].take(configuration['BATCHES_TO_PLOT']))
    return autoencoder_loss, l1_loss, l2_loss, ssim_score

In [ ]:
PREDICT_DATA_PATH = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask'


predict_data = {}  
for subset in ['predict_512']: # Folder with the subfolders NF/FF/PM
    nearfield = folder_to_dataloader(f'{PREDICT_DATA_PATH}/{subset}/beam_nf',configuration)
    farfield = folder_to_dataloader(f'{PREDICT_DATA_PATH}/{subset}/beam_ff',configuration)
    phasemask = folder_to_dataloader(f'{PREDICT_DATA_PATH}/{subset}/phasemask',configuration)
    predict_data[subset] = tf.data.Dataset.zip((nearfield, farfield, phasemask))

autoencoder_loss, l1_loss, l2_loss, ssim_score = predict(predict_data, model,configuration)
print(f"MAE: {l1_loss} | MSE: {l2_loss}| SSIM: {ssim_score}")


In [ ]:
from PIL import Image
import os

def reverse_phasemask(image_path):
    img = Image.open(image_path)
    inverted_img = Image.eval(img, lambda px: 255 - px)
    return inverted_img

image_path = r"\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\phasemask_predicted.png"
inverted_image = reverse_phasemask(image_path)

directory, filename = os.path.split(image_path)
inverted_image_path = os.path.join(directory, 'reverse_' + filename)

inverted_image.save(inverted_image_path)



Create compensatted phasemask

In [ ]:
from PIL import Image
import numpy as np
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.color import rgb2gray

In [ ]:
def uncrop_to_original(original_phasemask_path, cropped_image_path, save_path, crop_width=512, crop_height=512, rgb_value=128):
    original_image = Image.open(original_phasemask_path)
    cropped_image = Image.open(cropped_image_path)
    
    original_image_np = np.array(original_image)
    cropped_image_np = np.array(cropped_image)
    
    center_y, center_x = original_image_np.shape[0] // 2, original_image_np.shape[1] // 2

    start_x = max(center_x - crop_width // 2, 0)
    start_y = max(center_y - crop_height // 2, 0)
    
    end_x = min(start_x + crop_width, original_image_np.shape[1])
    end_y = min(start_y + crop_height, original_image_np.shape[0])
    
    new_image_np = np.full_like(original_image_np, rgb_value)
    
    new_image_np[start_y:end_y, start_x:end_x] = cropped_image_np
    
    new_image = Image.fromarray(new_image_np)
    
    new_image.save(save_path)
    
    return new_image


In [ ]:
# # Test of the uncrop_to_original
# original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\phasemask\phase_mask_1.png'
# phasemask_predicted_reverse = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\MSE_simulated\vortex_image_2\phasemask_predicted_reverse.png'
# phasemask_predicted_reverse_original_size = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\test.png'

# uncrop_to_original(original_phasemask_path,phasemask_predicted_reverse,phasemask_predicted_reverse_original_size)


In [ ]:
# resize the predicted phasemask back to original size
#predicted_phasemask_reverse_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\phasemask_predicted_reverse.png'
predicted_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\phasemask_predicted.png'
original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\refrence_phasemask.png' # can be one phasemask doesen't matter but needs the original size
predicted_phasemask_original_size_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\predicted_phasemask_original_size.png' 

# recrop the predicted_phasemask_reverse to the original size 800x600
predicted_phasemask_original_size = uncrop_to_original(original_phasemask_path,predicted_phasemask_path,predicted_phasemask_original_size_path)
print("Recroped the predicted reverse phasemask")

In [ ]:
predicted_phasemask_original_size_path= r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\predicted_phasemask_original_size.png'
original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Test\phasemask\phase_mask_1.png'

predicted_phasemask_original_size = cv2.imread(predicted_phasemask_original_size_path)
original_phasemask = cv2.imread(original_phasemask_path)

predicted_phasemask_original_size = rgb2gray(predicted_phasemask_original_size)
original_phasemask = rgb2gray(original_phasemask)

plt.title("Beam_cropped")
im=plt.imshow(predicted_phasemask_original_size,cmap='gray')
plt.colorbar(im)
plt.show()

plt.title("Beam_cropped")
im=plt.imshow(original_phasemask,cmap='gray')
plt.colorbar(im)
plt.show()

# print(predicted_phasemask_original_size.max())
# print(original_phasemask.max())
# print(predicted_phasemask_original_size.min())
# print(original_phasemask.min())

compensate_phasemask = abs(original_phasemask - predicted_phasemask_original_size)

# print(compensate_phasemask.max())
# print(compensate_phasemask.min())
#print(compensate_phasemask)

print(compensate_phasemask)

plt.title("compensate_phasemask")
im=plt.imshow(compensate_phasemask,cmap='gray')
plt.colorbar(im)
plt.show()


plt.title("compensate_phasemask")
plt.hist(compensate_phasemask)
plt.show()

print(compensate_phasemask.min())
print(compensate_phasemask.max())

compensate_phasemask_normalized_255 = (compensate_phasemask * 255)

plt.title("compensate_phasemask_normalized_255")
plt.hist(compensate_phasemask_normalized_255)
plt.show()

# save image
output_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\compensate_phasemask.png'
cv2.imwrite(output_path, compensate_phasemask_normalized_255)

In [ ]:
# # load compensate_phasemask_origininal_size and abbreation phasemask
# predicted_phasemask_reverse_original_size_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\predicted_phasemask_reverse_original_size.png' 
# original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\10013_idx_2024-06-25 21-47-29_2588_0.png'


# predicted_phasemask_reverse_original_size = cv2.imread(predicted_phasemask_reverse_original_size_path)
# original_phasemask = cv2.imread(original_phasemask_path)



# # check if the image are from [0:255] if not change them to that
# print(predicted_phasemask_reverse_original_size.max())
# print(original_phasemask.max())
# print(predicted_phasemask_reverse_original_size.min())
# print(original_phasemask.min())

# # compensate the phasemask original phasemask - predicted_phasemask_reverse_original_size
# # when i have two identical phasemask the image is black is this right ?

# compensate_phasemask = original_phasemask - predicted_phasemask_reverse_original_size

# # save image
# output_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\compensate_phasemask.png'
# cv2.imwrite(output_path, compensate_phasemask)

# X Images

In [ ]:
# def predict(data_tuple, autoencoder, configuration):
#     """Predict the Phasemask and output all metrics and the overview image."""
#     autoencoder_loss, l1_loss, l2_loss, ssim_score = evaluate(autoencoder, data_tuple, configuration)
#     return autoencoder_loss, l1_loss, l2_loss, ssim_score


In [ ]:
# import tensorflow as tf
# import pandas as pd

# PREDICT_DATA_PATH = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask'
# folder_test = 'predict_512'

# # Load the data
# predict_data = {}
# for subset in [folder_test]:  # Ordner mit den Unterordnern NF/FF/PM !
#     nearfield = folder_to_dataloader(f'{PREDICT_DATA_PATH}/{subset}/beam_nf', configuration)
#     farfield = folder_to_dataloader(f'{PREDICT_DATA_PATH}/{subset}/beam_ff', configuration)
#     phasemask = folder_to_dataloader(f'{PREDICT_DATA_PATH}/{subset}/phasemask', configuration)
#     predict_data[subset] = tf.data.Dataset.zip((nearfield, farfield, phasemask))

# results = []
# for data_tuple in predict_data[folder_test]:
#     autoencoder_loss, l1_loss, l2_loss, ssim_score = predict(predict_data[folder_test], model, configuration)
#     results.append({
#         "MSE": l1_loss,
#         "MAE": l2_loss,
#         "SSIM": ssim_score
#     })
#     print(f"MSE: {l1_loss} | MAE: {l2_loss} | SSIM: {ssim_score}")


# results_df = pd.DataFrame(results)
# results_df.to_csv('prediction_results.csv', index=False)
# print("Finished - Ergebnisse wurden in 'prediction_results.csv' gespeichert.")